In [1]:
from data_transformation import get_model_data,get_data_loaders,get_dataframes
from env_var import load_env_var
import torch
import numpy as np
from datetime import datetime
import random
import dotenv
from tf_classes import TFModel,BaselineMeanModel,BaselineZeroModel,BaselinePrevModel
from torch import nn,optim
import lightning as L
from Lightning_modules import LitDataModule,LitTFmodel
from lightning.pytorch import callbacks
import os
import json

In [22]:
dotenv.load_dotenv()
device='cuda'
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
results_dir=os.path.join(os.getcwd(),'data_dir','model',datetime.now().strftime('%d-%h-%y-%H%M/'))
os.makedirs(results_dir,exist_ok=True)
data_dir=load_env_var('MODEL_DATA_DIR','path')
file_count=1000
input_duration=50
pred_duration=1
window_inc=1
resolution=1000
batch_size=8
data_block_size=252
print(f'saving data to {results_dir}')
model_type='tf'


saving data to /workspace/pf_v2/data_dir/model/04-May-25-2233/


In [3]:
df_dict,files=get_dataframes('/workspace/pf_v2/data_dir/processed/250422_33.npz')
test_key=files[0]

filled datapoints due to NAs:
{'EMAMILTD': np.int64(0), 'MFSL': np.int64(0), 'SAREGAMA': np.int64(0), 'MARICO': np.int64(0), 'BASF': np.int64(0), 'SOLARINDS': np.int64(0), 'DCMSHRIRAM': np.int64(0), 'TITAGARH': np.int64(0), 'COFORGE': np.int64(0), 'BAJAJHLDNG': np.int64(0), 'BRITANNIA': np.int64(0), 'AMBER': np.int64(0), 'HINDZINC': np.int64(0), 'MAHSCOOTER': np.int64(0), 'JUSTDIAL': np.int64(0), 'ASTERDM': np.int64(0), 'THERMAX': np.int64(0), 'INGERRAND': np.int64(0), 'GODFRYPHLP': np.int64(0), 'CHOLAHLDNG': np.int64(0), 'SUPREMEIND': np.int64(0), 'GLENMARK': np.int64(0), 'BHARTIARTL': np.int64(0), 'ENDURANCE': np.int64(0), 'IFCI': np.int64(0), 'VGUARD': np.int64(0), 'FORTIS': np.int64(0), 'HEG': np.int64(0), 'RVNL': np.int64(0), 'ASAHIINDIA': np.int64(0), 'COLPAL': np.int64(0), 'INFY': np.int64(0), 'INDUSTOWER': np.int64(0)}
(1372, 2) EMAMILTD
 rows dropped due to extra length:0 
 rows dropped due to NA counts: 0

(1372, 2) MFSL
 rows dropped due to extra length:0 
 rows dropped due 

In [4]:
close_prices={}
for i in df_dict.keys():
    close_prices[i],_=(df_dict[i][['close']][:]),df_dict[i].drop(columns='close',axis=1,inplace=True)

In [5]:
(xin,yop)=get_model_data(df_dict,None,None,input_duration,pred_duration,window_inc)
(x_price,_)=get_model_data(close_prices,None,None,input_duration,0,window_inc)[-2:]
xin,x_price,yop=torch.tensor(xin),torch.tensor(x_price),torch.tensor(yop)
xin.shape,yop.shape,x_price.shape


(torch.Size([1322, 50, 33]),
 torch.Size([1322, 33, 1]),
 torch.Size([1323, 50, 33]))

In [6]:
datamod=LitDataModule(xin,yop,input_duration,pred_duration,data_block_size,batch_size)

In [7]:
stockvalloader,stocktrainloader,stocktestloader,data_splits=get_data_loaders(
    xin,yop,
    input_duration=input_duration,batch_size=batch_size,
    pred_duration=pred_duration,data_block_size=data_block_size,
    val_data_split=0.1,shuffle=True,
)


[dataIndices(val_start=62, val_end=72, train_start=122, train_end=204, test_start=254, test_end=264), dataIndices(val_start=264, val_end=274, train_start=324, train_end=406, test_start=456, test_end=466), dataIndices(val_start=466, val_end=476, train_start=526, train_end=608, test_start=658, test_end=668), dataIndices(val_start=668, val_end=678, train_start=728, train_end=810, test_start=860, test_end=870), dataIndices(val_start=870, val_end=880, train_start=930, train_end=1012, test_start=1062, test_end=1072)]


/workspace/pf_v2/data_transformation.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.vars=[torch.tensor(i) for i in args]


In [8]:
datamod.setup('fit')

[dataIndices(val_start=62, val_end=72, train_start=122, train_end=204, test_start=254, test_end=264), dataIndices(val_start=264, val_end=274, train_start=324, train_end=406, test_start=456, test_end=466), dataIndices(val_start=466, val_end=476, train_start=526, train_end=608, test_start=658, test_end=668), dataIndices(val_start=668, val_end=678, train_start=728, train_end=810, test_start=860, test_end=870), dataIndices(val_start=870, val_end=880, train_start=930, train_end=1012, test_start=1062, test_end=1072)]
[dataIndices(val_start=62, val_end=72, train_start=122, train_end=204, test_start=254, test_end=264), dataIndices(val_start=264, val_end=274, train_start=324, train_end=406, test_start=456, test_end=466), dataIndices(val_start=466, val_end=476, train_start=526, train_end=608, test_start=658, test_end=668), dataIndices(val_start=668, val_end=678, train_start=728, train_end=810, test_start=860, test_end=870), dataIndices(val_start=870, val_end=880, train_start=930, train_end=1012,

In [9]:
(next(iter(datamod.predict_dataloader())))

[tensor([[[ 2.8279e-02,  8.7086e-03, -8.1244e-04,  ...,  8.8350e-03,
            3.4880e-03, -4.6189e-02],
          [ 7.5075e-03,  1.9363e-03, -9.8006e-03,  ...,  8.0919e-03,
            1.4462e-02,  1.4680e-02],
          [ 5.7177e-03,  2.5532e-02, -3.8767e-03,  ...,  3.1375e-03,
           -1.1765e-02,  4.4618e-02],
          ...,
          [ 1.4097e-02,  3.7009e-02, -1.2709e-03,  ..., -1.3624e-02,
           -6.6703e-03,  1.2539e-03],
          [ 2.0973e-03, -1.4276e-02,  4.5470e-02,  ...,  5.9018e-03,
           -1.4765e-03,  0.0000e+00],
          [-4.3396e-03,  1.2056e-01,  3.3260e-02,  ..., -3.6361e-03,
           -9.8650e-03, -6.2854e-03]],
 
         [[ 7.5075e-03,  1.9363e-03, -9.8006e-03,  ...,  8.0919e-03,
            1.4462e-02,  1.4680e-02],
          [ 5.7177e-03,  2.5532e-02, -3.8767e-03,  ...,  3.1375e-03,
           -1.1765e-02,  4.4618e-02],
          [ 1.0112e-02, -1.7016e-02,  1.6112e-02,  ...,  1.4997e-02,
           -2.6255e-03,  1.8112e-02],
          ...,
    

In [10]:
base_zero=BaselineZeroModel(len(files),pred_duration)
base_prev=BaselinePrevModel(len(files),pred_duration,his_dim=1)
base_mean=BaselineMeanModel(len(files),pred_duration,mean_dim=1)
model_params={
    "d_model":8,
    "n_head":8,
    "files":files,
    "num_layers":6,
    "y_inp":False,
    "resolution":resolution,
    "pred_duration":1,
    "resolution": resolution,
    "input_duration":input_duration,
    "n_features":1,
    "reduce_dims":False
    } 


In [11]:
with torch.no_grad():
    print(base_zero(xin[:1]).shape,
          base_prev(xin[:1]).shape,
          base_mean(xin[:1]).shape,
          yop[:1].shape
          )


torch.Size([1, 33, 1]) torch.Size([1, 33, 1]) torch.Size([1, 33, 1]) torch.Size([1, 33, 1])


In [12]:
model=TFModel(**model_params)
model.eval()
with torch.no_grad():
    t=next(iter(datamod.predict_dataloader()))[0].to('cpu')
    print(model(t).shape)

model

torch.Size([8, 33, 1])


TFModel(
  (enc): Sequential(
    (input flatten): Flatten(start_dim=1, end_dim=-1)
    (embedding layer): PositionalEmbedding(
      (position): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (embedding): Embedding(2001, 8)
      (converter): ConvLogEmb()
    )
    (tf flatten): Flatten(start_dim=1, end_dim=-1)
    (reshape layer): Unflatten(dim=1, unflattened_size=(50, 264))
    (encoder layers): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=264, out_features=264, bias=True)
          )
          (linear1): Linear(in_features=264, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=264, bias=True)
          (norm1): LayerNorm((264,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((264,), ep

In [13]:
loss=nn.L1Loss
Litmodel=LitTFmodel(model=model,loss=loss,opt=torch.optim.Adam,stocks=files,reduction='mean')
print(Litmodel.loss)
Litmodel(next(iter(datamod.val_dataloader())),0).shape

L1Loss()


torch.Size([8, 33, 1])

In [14]:
trainer=L.Trainer(
    callbacks=[
        callbacks.EarlyStopping(f"avg_{Litmodel.loss}"[:-2])
    ],
    max_epochs=100,
    enable_progress_bar=True,
    enable_model_summary=True,
    enable_checkpointing=False,
    num_sanity_val_steps=0,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [15]:
trainer.fit(Litmodel,datamod)#train_dataloaders=datamod.train_dataloader(),val_dataloaders=datamod.val_dataloader()

You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | TFModel | 18.2 M | eval 
1 | loss  | L1Loss  | 0      | train
------------------------------------------
18.2 M    Trainable params
0         Non-trainable params
18.2 M    Total params
72.643    Total estimated model params size (MB)
1         Modules in train mode
173       Modules in eval mode


[dataIndices(val_start=62, val_end=72, train_start=122, train_end=204, test_start=254, test_end=264), dataIndices(val_start=264, val_end=274, train_start=324, train_end=406, test_start=456, test_end=466), dataIndices(val_start=466, val_end=476, train_start=526, train_end=608, test_start=658, test_end=668), dataIndices(val_start=668, val_end=678, train_start=728, train_end=810, test_start=860, test_end=870), dataIndices(val_start=870, val_end=880, train_start=930, train_end=1012, test_start=1062, test_end=1072)]
[dataIndices(val_start=62, val_end=72, train_start=122, train_end=204, test_start=254, test_end=264), dataIndices(val_start=264, val_end=274, train_start=324, train_end=406, test_start=456, test_end=466), dataIndices(val_start=466, val_end=476, train_start=526, train_end=608, test_start=658, test_end=668), dataIndices(val_start=668, val_end=678, train_start=728, train_end=810, test_start=860, test_end=870), dataIndices(val_start=870, val_end=880, train_start=930, train_end=1012,

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [16]:
trainer.validate(Litmodel,datamod)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[dataIndices(val_start=62, val_end=72, train_start=122, train_end=204, test_start=254, test_end=264), dataIndices(val_start=264, val_end=274, train_start=324, train_end=406, test_start=456, test_end=466), dataIndices(val_start=466, val_end=476, train_start=526, train_end=608, test_start=658, test_end=668), dataIndices(val_start=668, val_end=678, train_start=728, train_end=810, test_start=860, test_end=870), dataIndices(val_start=870, val_end=880, train_start=930, train_end=1012, test_start=1062, test_end=1072)]
[dataIndices(val_start=62, val_end=72, train_start=122, train_end=204, test_start=254, test_end=264), dataIndices(val_start=264, val_end=274, train_start=324, train_end=406, test_start=456, test_end=466), dataIndices(val_start=466, val_end=476, train_start=526, train_end=608, test_start=658, test_end=668), dataIndices(val_start=668, val_end=678, train_start=728, train_end=810, test_start=860, test_end=870), dataIndices(val_start=870, val_end=880, train_start=930, train_end=1012,

Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Validate metric         ┃          DataLoader 0           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           avg_L1Loss            │       0.01603734585735238       │
│      avg_MeanAbsoluteError      │       0.01603734585735238       │
│ avg_MeanAbsolutePercentageError │        42.25394873538421        │
│      avg_MeanSquaredError       │      0.0005177491977704608      │
└─────────────────────────────────┴─────────────────────────────────┘

[{'avg_L1Loss': 0.01603734585735238,
  'avg_MeanAbsoluteError': 0.01603734585735238,
  'avg_MeanAbsolutePercentageError': 42.25394873538421,
  'avg_MeanSquaredError': 0.0005177491977704608}]

In [17]:
trainer.test(Litmodel,datamod)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[dataIndices(val_start=62, val_end=72, train_start=122, train_end=204, test_start=254, test_end=264), dataIndices(val_start=264, val_end=274, train_start=324, train_end=406, test_start=456, test_end=466), dataIndices(val_start=466, val_end=476, train_start=526, train_end=608, test_start=658, test_end=668), dataIndices(val_start=668, val_end=678, train_start=728, train_end=810, test_start=860, test_end=870), dataIndices(val_start=870, val_end=880, train_start=930, train_end=1012, test_start=1062, test_end=1072)]
[dataIndices(val_start=62, val_end=72, train_start=122, train_end=204, test_start=254, test_end=264), dataIndices(val_start=264, val_end=274, train_start=324, train_end=406, test_start=456, test_end=466), dataIndices(val_start=466, val_end=476, train_start=526, train_end=608, test_start=658, test_end=668), dataIndices(val_start=668, val_end=678, train_start=728, train_end=810, test_start=860, test_end=870), dataIndices(val_start=870, val_end=880, train_start=930, train_end=1012,

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃           Test metric           ┃          DataLoader 0           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           avg_L1Loss            │      0.015922639944315496       │
│      avg_MeanAbsoluteError      │      0.015922639944315496       │
│ avg_MeanAbsolutePercentageError │       19.664987535244496        │
│      avg_MeanSquaredError       │      0.0005336959862481697      │
└─────────────────────────────────┴─────────────────────────────────┘

[{'avg_L1Loss': 0.015922639944315496,
  'avg_MeanAbsoluteError': 0.015922639944315496,
  'avg_MeanAbsolutePercentageError': 19.664987535244496,
  'avg_MeanSquaredError': 0.0005336959862481697}]

In [18]:
prices=np.array([i['close'][-2:-1] for i in close_prices.values()])

In [25]:
with open(os.path.join(results_dir,'live_predictions.json'),'w') as f:
    predictions={x:y for x,y in zip(files,((1+Litmodel((xin[-2:-1],yop[-1]),0).detach().numpy())*prices).squeeze())}
    print(predictions)
    json.dump(predictions,f)

{'EMAMILTD': np.float64(590.1508803839245), 'MFSL': np.float64(1140.0916327238083), 'SAREGAMA': np.float64(497.45161332491966), 'MARICO': np.float64(630.970198908344), 'BASF': np.float64(5632.013599467755), 'SOLARINDS': np.float64(10620.509427576908), 'DCMSHRIRAM': np.float64(1080.07796895504), 'TITAGARH': np.float64(1318.0095558553028), 'COFORGE': np.float64(9444.098011341062), 'BAJAJHLDNG': np.float64(11293.486869335175), 'BRITANNIA': np.float64(4788.23546308279), 'AMBER': np.float64(6056.280419480056), 'HINDZINC': np.float64(488.22192622761355), 'MAHSCOOTER': np.float64(9708.606566598406), 'JUSTDIAL': np.float64(1026.4683707363233), 'ASTERDM': np.float64(492.3489516438133), 'THERMAX': np.float64(4768.950462824316), 'INGERRAND': np.float64(4430.222253026441), 'GODFRYPHLP': np.float64(5888.177425353613), 'CHOLAHLDNG': np.float64(1516.208613716825), 'SUPREMEIND': np.float64(4964.180449336767), 'GLENMARK': np.float64(1515.9785652811431), 'BHARTIARTL': np.float64(1615.8944777549136), 'EN